# Compute and compare different radial potentials

In [1]:
using JAC

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b5211b19a5aa56f2c76a4f66df0ec7b35773f4c6-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/b5211b19a5aa56f2c76a4f66df0ec7b35773f4c6-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")


We shall here describe how different atomic (radial) potentials can be calculated and applied within the JAC toolbox.

Any (radial) potential is handled in JAC by the data struct that (may) contain a name the radial function and the radial grid on which the potential has been defined. The potential function V typically contains the effective charge Z(r) as seen by an electron, so that the full radial potential is given by -Z(r) / r. This convention has been frequently applied in atomic physics and has been adopted in JAC.


In [2]:
? Radial.Potential

`struct  Radial.Potential`  ... defines a struct for the radial potential which contains all information about its physical content.

```
+ name           ::String            ... A name for the potential.
+ V              ::Array{Float64,1}  ... radial potential function.
+ grid           ::RadialGrid        ... radial grid on which the potential is defined.
```

---

`JAC.Radial.Potential()`  ... constructor to define an 'empty' instance of the radial potential.



Perhaps, the simplest choice is the **nuclear** potential which, of course, depends on the nuclear model.
We consider a ^12C carbon atom with a Fermi-distributed nuclear charge and a standard radial grid.


In [3]:
grid     = JAC.Radial.Grid("grid: exponential")
nucModel = Nuclear.Model(6., "Fermi")

Define a radial grid of type MeshGL with 400 grid points
 [rnt=2.000e-06, h=5.000e-02, hp=0.000e+00, NoPoints=390, r_max=9.161e+02;
  B-splines wit break points at every 7th point, nsL=56, nsS=57, orderL=7, orderS=8, orderGL=7] 


Fermi nuclear model for Z = 12.0 with mass = 24.72, radius R = 3.0053184906569026 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0.


Then, the nuclear potential is generated by simply calling:


In [4]:
nucPotential = JAC.Nuclear.nuclearPotential(nucModel, grid)

nuclear-potential: Fermi-distributed (radial) potential ... defined on 400 grid points ...
V:    [0.0, 0.00578622, 0.0293868, 0.0675525, 0.113694, 0.159833, 0.197995, 0.221591, 0.235586, 0.269068  …  0.508202, 0.541656, 0.561496, 0.608957, 0.685688, 0.778412, 0.871083, 0.94769, 0.995039, 1.02311]  ...  [12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0  …  12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 12.0]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = MeshGL, ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...  [842.031, 887.781, 916.071]
wr:   [0.0, 5.42629e-8, 1.17215e-7]  ...  [52.0386, 38.1203, 17.6472]
t:    [0.0, 0.0, 0.0]  ...  [39.5528, 56.1281, 79.6496]


As seen from the output, the effective charge Z(r) vanishes near to the origin but quickly increases to the full charge Z = 6. at larger r-values, i.e. outside of the nucleus

To calculate **atomic** potentials, we also need to know the electron density around the nuclues. Most often, this density refers to a particular atomic level. Therefore, we first need to compute some of the low-lying levels of, for instance, neutral carbon. We shall here just compute the levels of the 1s^2 2s^2 2p^2 ground-state configuration. We use the *simplest* approximation for these computation but will explain in some later tutorials how we could control and refine such simple structure calculations, if needed.


In [5]:
basis     = perform("computation: SCF", [Configuration("1s^2 2s^2 2p^2")], nucModel, grid, AsfSettings())
multiplet = perform("computation: CI",  basis, nucModel, grid, AsfSettings());


... in perform('computation: SCF', ...
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_3/2^2 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^1 2p_3/2^1 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 
(Re-) Define a new standard subshell list.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 12.0 with mass = 24.72, radius R = 3.0053184906569026 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -7.21385124e+01    -7.21385593e+01    +6.50023285e-07    
      2      2s_1/2      -1.80433085e+01    -1.80433081e+01    -2.12646900e-08    
      3      3s_1/2      -8.01545204e+00    -8

  2p_1/2::  en [a.u.] = -7.37490611e+00;   self-consistency = 7.475644e-07  [1.678560e-06 for symmetry block kappa = 1]
  2p_3/2::  en [a.u.] = -7.35877586e+00;   self-consistency = 6.593220e-07  [1.099936e-06 for symmetry block kappa = -2]

Iteration 7 for symmetries ... 
  1s_1/2::  en [a.u.] = -5.31901532e+01;   self-consistency = 1.298894e-07  [3.935443e-07 for symmetry block kappa = -1]
  2s_1/2::  en [a.u.] = -8.38762750e+00;   self-consistency = 2.161652e-07  [3.935443e-07 for symmetry block kappa = -1]
  2p_1/2::  en [a.u.] = -7.37490760e+00;   self-consistency = 1.010294e-07  [2.261775e-07 for symmetry block kappa = 1]
  2p_3/2::  en [a.u.] = -7.35877692e+00;   self-consistency = 7.259520e-08  [1.203242e-07 for symmetry block kappa = -2]
Compute CI matrix of dimension 1 x 1 for the symmetry 1^+ ...   ... done.
Compute CI matrix of dimension 2 x 2 for the symmetry 2^+ ...   ... done.
Compute CI matrix of dimension 2 x 2 for the symmetry 0^+ ...   ... done.

  Eigenenergies:

  


From this multiplet, we just take the *lowest* level (the ground level) in order to calculate the density for the (electronic) potential. We have moreover different choices to generate such an **electronic** potential as seen from ? compute at the REPL:

    •    ("radial potential: core-Hartree", grid::Radial.Grid, level::Level) ... to compute a (radial) 
           core-Hartree potential from the given list of orbitals; cf. JAC.computePotentialCoreHartree. 
           A potential::RadialPotential is returned. 

    •    ("radial potential: Kohn-Sham", grid::Radial.Grid, level::Level) ... to compute a (radial) 
           Kohn-Sham potential from the given list of orbitals; cf. JAC.computePotentialKohnSham. 
           A potential::RadialPotential is returned.

    •    ("radial potential: Dirac-Fock-Slater", grid::Radial.Grid, level::Level) ... to compute a (radial) 
          Dirac-Fock-Slater potential from the given list of orbitals; this potential is rather simple 
          but includes some undesired self-interaction and exhibits an asymptotic behaviour.         
          Cf. JAC.computePotentialDFS. A potential::RadialPotential is returned.


In [6]:
level  = multiplet.levels[1]
potCH  = compute("radial potential: core-Hartree", grid, level)
potDFS = compute("radial potential: Dirac-Fock-Slater", grid, level)

coreSubshells = Subshell[1s_1/2, 2s_1/2];    subshells = Subshell[1s_1/2, 2s_1/2, 2p_1/2, 2p_3/2]


DFS (radial) potential ... defined on 400 grid points ...
V:    [0.0, -4.70521e-7, -2.42189e-6, -5.56237e-6, -9.35469e-6, -1.3151e-5, -1.62941e-5, -1.82383e-5, -1.93915e-5, -2.21502e-5  …  -4.18389e-5, -4.45938e-5, -4.62279e-5, -5.01384e-5, -5.64643e-5, -6.41149e-5, -7.17665e-5, -7.80951e-5, -8.20079e-5, -8.43283e-5]  ...  [-5.99995, -5.99995, -5.99995, -5.99996, -5.99996, -5.99997, -5.99998, -5.99999, -5.99999, -5.99999  …  -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = MeshGL, ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...  [842.031, 887.781, 916.071]
wr:   [0.0, 5.42629e-8, 1.17215e-7]  ...  [52.0386, 38.1203, 17.6472]
t:    [0.0, 0.0, 0.0]  ...  [39.5528, 56.1281, 79.6496]


All these potentials are represented as *effective charge* Z(r) and can be added to the nuclear potential, for example:

In [7]:
JAC.add(nucPotential, potDFS)

nuclear-potential: Fermi-distributed+nuclear-potential: Fermi-distributed (radial) potential ... defined on 400 grid points ...
V:    [0.0, 0.00578575, 0.0293844, 0.067547, 0.113685, 0.15982, 0.197978, 0.221573, 0.235566, 0.269046  …  0.50816, 0.541612, 0.561449, 0.608907, 0.685631, 0.778348, 0.871012, 0.947612, 0.994957, 1.02303]  ...  [6.00005, 6.00005, 6.00005, 6.00004, 6.00004, 6.00003, 6.00002, 6.00001, 6.00001, 6.00001  …  6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = MeshGL, ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...  [842.031, 887.781, 916.071]
wr:   [0.0, 5.42629e-8, 1.17215e-7]  ...  [52.0386, 38.1203, 17.6472]
t:    [0.0, 0.0, 0.0]  ...  [39.5528, 56.1281, 79.6496]


What can be seen ... and how to display these potentials also graphically; use JAC.plot()